<a href="https://colab.research.google.com/github/ChandanShrivastava/mpi_matrixmultiplication/blob/master/MatrixMultiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Example usage:
import time
import numpy as np

In [4]:
leng = 300
matrix_a = np.random.randint(0, 1000, size=(leng, leng)).tolist()
matrix_b = np.random.randint(0, 1000, size=(leng, leng)).tolist()

print("Matrix A:")
#for row in matrix_a:
     #print(row)
#
print("\nMatrix B:")
#for row in matrix_b:
     #print(row)

Matrix A:

Matrix B:


In [5]:
def multiply_matrices(matrix1, matrix2):
    # Get dimensions
    rows1 = len(matrix1)
    cols1 = len(matrix1[0])
    rows2 = len(matrix2)
    cols2 = len(matrix2[0])

    # Check if multiplication is possible
    if cols1 != rows2:
        return "Error: Number of columns in the first matrix must equal the number of rows in the second matrix."

    # Initialize result matrix with zeros
    result_matrix = [[0 for _ in range(cols2)] for _ in range(rows1)]

    # Perform multiplication
    for i in range(rows1):
        for j in range(cols2):
            for k in range(cols1):
                result_matrix[i][j] += matrix1[i][k] * matrix2[k][j]

    return result_matrix



In [11]:
# Start the timer
start_time = time.time()

product = multiply_matrices(matrix_a, matrix_b)

if isinstance(product, str):
    print(product)
else:
    #for row in product:
    print(len(product))

# Stop the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print(f"Execution time for a normal run: {execution_time:.6f} seconds")

300
Execution time for a normal run: 4.192351 seconds


In [ ]:
# Install the mpi4py package
!pip -qq install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import sys
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np

In [ ]:
%%writefile parallel_multiply_matrices_mpi.py


def parallel_multiply_matrices_mpi(matrix1, matrix2):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Basic dimension check (can be made more robust)
    if len(matrix1[0]) != len(matrix2):
        if rank == 0:
            print("Error: Number of columns in the first matrix must equal the number of rows in the second matrix.")
        return None

    rows1 = len(matrix1)
    cols1 = len(matrix1[0])
    rows2 = len(matrix2)
    cols2 = len(matrix2[0])

    # Only the root process handles the initial data and gathers results
    if rank == 0:
        matrix_a = np.array(matrix1)
        matrix_b = np.array(matrix2)
        result_matrix = np.empty((rows1, cols2), dtype=int)

        # Distribute rows of matrix_a to other processes
        rows_per_process = rows1 // size
        remainder = rows1 % size

        for i in range(size):
            start_row = i * rows_per_process + min(i, remainder)
            end_row = start_row + rows_per_process + (1 if i < remainder else 0)
            if i == 0:
                # The root process works on its assigned rows
                local_matrix_a = matrix_a[start_row:end_row, :]
            else:
                # Send rows to other processes
                comm.send(matrix_a[start_row:end_row, :], dest=i, tag=11)
                # Send matrix_b to other processes
                comm.send(matrix_b, dest=i, tag=22)

    else:
        # Receive rows of matrix_a and matrix_b
        local_matrix_a = comm.recv(source=0, tag=11)
        matrix_b = comm.recv(source=0, tag=22)
        result_matrix = None # Only root process holds the final result array

    # Each process performs multiplication on its assigned rows
    if local_matrix_a is not None:
        local_result = np.dot(local_matrix_a, matrix_b)
    else:
        local_result = None

    # Gather results from all processes to the root
    comm.Gather(local_result, result_matrix, root=0)

    if rank == 0:
        return result_matrix
    else:
        return None


if __name__ == "__main__":
    # Check if there are enough command-line arguments
    if len(sys.argv) == 2:
        # The script name is sys.argv[0], so arguments start from index 1
        matrix_a = sys.argv[1]
        matrix_b = sys.argv[2]
        product = parallel_multiply_matrices_mpi(matrix_a, matrix_b)
        if MPI.COMM_WORLD.Get_rank() == 0 and product is not None:
            print(product)
    else:
        print("Usage: python3 script.py matrix_a matrix_b")


Writing parallel_multiply_matrices_mpi.py


In [ ]:
# Start the timer
start_time = time.time()

!mpirun --allow-run-as-root --oversubscribe -np 2 python parallel_multiply_matrices_mpi.py matrix_a matrix_b

# Stop the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print(f"Execution time for a MPI run: {execution_time:.6f} seconds")


Execution time for a MPI run: 0.103842 seconds


SyntaxError: invalid syntax (<timeit-src>, line 3)